In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from code_vi.system import OpticsManager
from code_vi.elements import OpticalElement
from code_vi.ray_trace import RayTracer
from code_vi.visualization import Draw


# 1. Initialize
manager = OpticsManager()

# Lens: Positioned at x=12.7, y=50, Facing Down (90 deg)
lens1 = OpticalElement(
    name="Lens 1", optic_type="Lens",
    x_center=12.7, y_center=250, 
    orientation_angle=90, clear_aperture=22.86,
    diameter=25.4, center_thickness=2.40,R1=np.inf, R2=-350.67,
    material="ZnSe"
)
lens2 = OpticalElement(
    name="Lens 2", optic_type="Lens",
    x_center=12.7, y_center=750, 
    orientation_angle=90, clear_aperture=22.86,
    diameter=25.4, center_thickness=2.40,R1=350.67, R2=np.inf,
    material="ZnSe"
)

focal_plane_offset = 25.0 
    
# Grating 2a (75 l/mm)
grating1 = OpticalElement(
    name="Grating 1 (75)", optic_type="Grating",
    x_center=12.7, y_center=1000 + focal_plane_offset, 
    orientation_angle=-88.0,   
    clear_aperture=45, diameter=50.8,
    groove_density=75.0, diffraction_order=-1, material="Gold"
)

# Grating 2b (25 l/mm)
# Maintain the relative 35mm distance from G2a
grating2 = OpticalElement(
    name="Grating 2 (25)", optic_type="Grating",
    x_center=-12.7, y_center=(1000 + focal_plane_offset) - 35.0, 
    orientation_angle=0,   
    clear_aperture=45, diameter=50.8,
    groove_density=25.0, diffraction_order=1, material="Gold" # Keep order matching to add dispersion
)

manager.add_element(lens1)
manager.add_element(lens2)
manager.add_element(grating1)
manager.add_element(grating2)


tracer = RayTracer(manager)

# --- SMART GENERATION ---
# Automatically finds the valid grating region and optimizes angles
tracer.generate_smart_spr_source(
    n_sources=11,                # Number of distinct source points you want
    rays_per_source=20,         # Rays per point (uniformly distributed in valid cone)
    target_optic_name="Lens 2", # The optic that defines "success" (can be OAP/Mirror too)
    grating_search_bounds=(0, 25.4), # Max physical length of grating to scan
    acceptance_angle_range=(70, 110), 
    grating_period=10.0,
    beam_energy=0.99
)    

# 3. Run Simulation
print("Running Simulation...")
for t in np.arange(0, 5500, 50.0):
    tracer.run_time_step(t, 50.0)
tracer._sync_to_dataframe()

# 4. VISUALIZATION (Now handled entirely by the class)
# This automatically handles the slider, the figure creation, 
# and preserving your zoom level when switching sources.
Draw.interactive_session(
manager, 
tracer, 
show_curvature=False, 
show_skeleton=True,       # <--- You can toggle these easily now
draw_beam_arrow=True,
show_intersection_points=False
)


--- Smart Source Generation (Target: Lens 2) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 1.34mm to 24.06mm (Extent: 22.73mm)
   Step 2: Optimizing angles for 11 sources...
